# Global preparations

In [1]:
import sys
import os
import matplotlib.pyplot as plt
import time
import torch
import torchvision
import logging
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision.transforms.functional import to_pil_image
from PIL import Image
from tqdm.notebook import tqdm

sys.path.insert(0, os.path.expanduser('~/spiky'))
device = 'cuda:7'
summation_dtype = torch.float32
random_seed = 1
torch.manual_seed(random_seed)
np.random.seed(random_seed)
torch.backends.cudnn.enabled = True
logger = logging.getLogger('stdout_logger')
logger.setLevel(logging.INFO)
if not logger.hasHandlers():
    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setFormatter(logging.Formatter(fmt='%(asctime)s|%(levelname)s|%(message)s'))
    logger.addHandler(stdout_handler)

# MNIST preparation

Let's read the MNIST dataset and represent it in sparse form

In [2]:
mnist_dataset_dir = 'mnist'
mnist_train_dataset = torchvision.datasets.MNIST(
    mnist_dataset_dir, train=True, download=True
)
mnist_test_dataset = torchvision.datasets.MNIST(
    mnist_dataset_dir, train=False, download=True
)
mnist_train_data_device = mnist_train_dataset.data.to(device=device).to(dtype=torch.float32) / 255
mnist_test_data_device = mnist_test_dataset.data.to(device=device).to(dtype=torch.float32) / 255
mnist_train_targets_device = mnist_train_dataset.targets.to(device=device)
mnist_test_targets_device = mnist_test_dataset.targets.to(device=device)

mnist_train_dataset = TensorDataset(
    mnist_train_data_device, mnist_train_targets_device
)
mnist_test_dataset = TensorDataset(
    mnist_test_data_device, mnist_test_targets_device
)

In [3]:
mnist_test_loader = torch.utils.data.DataLoader(
    mnist_train_dataset,
    batch_size=1, shuffle=True
)

example_data, example_targets = next(iter(mnist_test_loader))
to_pil_image(example_data[0])

In [4]:
patch_h = patch_w = 28
input_shape = (patch_h, patch_w)
clustering_dim = 44

In [5]:
example_data.shape

torch.Size([1, 28, 28])

In [6]:
# HYPERPLANE preprocessing
# # x = example_data
# # x = x.reshape(x.shape[0], 28 * 28)
# # m = (torch.rand(28 * 28, 2048 * 10, device=device) - 0.5) * 2
# # (x @ m).shape
# # m[:,torch.arange(0, 1024 * 10) * 2 + 1] = 0
# mnist_train_data_device = mnist_train_data_device.reshape(mnist_train_data_device.shape[0], 28 * 28) @ m
# mnist_test_data_device = mnist_test_data_device.reshape(mnist_test_data_device.shape[0], 28 * 28) @ m
# mnist_train_dataset = TensorDataset(
#     mnist_train_data_device, mnist_train_targets_device
# )
# mnist_test_dataset = TensorDataset(
#     mnist_test_data_device, mnist_test_targets_device
# )

# Baseline MLP

In [7]:
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

class BaselineMLP(nn.Module):
    def __init__(self, device, n_inputs, hidden_dim):
        super().__init__()
        self.fc1 = nn.Linear(n_inputs, hidden_dim, bias=False, device=device)
        self.fc2 = nn.Linear(hidden_dim, 10, bias=False, device=device)
        
    def forward(self, x):
        x1 = x / (x.norm(dim=-1, keepdim=True) + 1e-16)
        x2 = F.relu(self.fc1(x1))
        return self.fc2(x2)

In [ ]:
baseline_net = BaselineMLP(device, input_shape[0] * input_shape[1], clustering_dim**2)

In [ ]:
optimizer = optim.Adam(baseline_net.parameters(), lr=0.001)

loss_func = nn.CrossEntropyLoss()
for epoch in range(16):
    baseline_net.train()
    train_loader = torch.utils.data.DataLoader(mnist_train_dataset, batch_size=256, shuffle=True)
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        target_one_hot = torch.nn.functional.one_hot(target, num_classes=10)
        optimizer.zero_grad()
        output = baseline_net(data.reshape([data.shape[0], input_shape[0] * input_shape[1]]))
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    train_acc = 100. * correct / len(train_loader.dataset)
    
    baseline_net.eval()
    correct = 0
    with torch.no_grad():
        test_loader = torch.utils.data.DataLoader(mnist_test_dataset, batch_size=256, shuffle=True)
        for data, target in test_loader:
            output = baseline_net(data.reshape([data.shape[0], input_shape[0] * input_shape[1]]))
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_acc = 100. * correct / len(test_loader.dataset)
    print(f'Epoch {epoch+1} Train/test accuracy: {train_acc:.2f}/{test_acc:.2f}')

# Simple LUT network

In [23]:
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

from spiky.lut.LUTLayer import (
    Conv2DLUTLayer, LUTLayer, LUTSharedContext, SynapseMeta, GradientPolicy, GradientType
)
from spiky.util.torch_utils import make_lr_getter

synapse_meta = SynapseMeta(
    min_weight=-1.0,
    max_weight=1.0,
    initial_weight=0.0,
    initial_noise_level=0.0
)

shared_lut_ctx = LUTSharedContext()
shared_lut_ctx.to_device(device)
g_policy = GradientPolicy(GradientType.Internal, normalized=False)

lut_net = None
baseline_net = None
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [28]:
class LutNet(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.filter_lut = LUTLayer(
            n_inputs=28 * 28,
            n_anchors_per_detector=10,
            n_detectors=2048,
            n_outputs=128,
            synapse_meta=synapse_meta,
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            summation_dtype=summation_dtype,
            # _explicit_anchors=torch.arange(0, 20480, dtype=torch.int32), # for hyperplanes
            random_seed=random_seed,
            device=device
        )
        print('filter finished')
        l = []
        for i in range(4):
            l.append(
                LUTLayer(
                    n_inputs=128,
                    n_anchors_per_detector=15,
                    n_detectors=128,
                    n_outputs=128,
                    synapse_meta=synapse_meta,
                    weights_gradient_policy=g_policy,
                    shared_context=shared_lut_ctx,
                    summation_dtype=summation_dtype,
                    random_seed=random_seed,
                    device=device
                )
            )
            print(f'conv_lut {i} finished')
        self.conv_luts = nn.ModuleList(l)
        self.final_lut = LUTLayer(
            n_inputs=128,
            n_anchors_per_detector=15,
            n_detectors=128,
            n_outputs=10,
            synapse_meta=synapse_meta,
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            device=device
        ) 

    def forward(self, x):
        B = x.shape[0]
        x = x.reshape(B, 1, 28 * 28)
        x /= x.norm(dim=-1, keepdim=True)
        x = self.filter_lut(x.reshape(B, 1, 28 * 28))
        # x = self.filter_lut(x)
        for c_lut in self.conv_luts:
            x = c_lut(x) + x
        return self.final_lut(x).squeeze(1)
    
    def setup_external_learning_rate_hook(self, optimizer):
        lr_getter = make_lr_getter(optimizer)
        self.filter_lut.set_external_learning_rate_hook(lr_getter)
        for c_lut in self.conv_luts:
            c_lut.set_external_learning_rate_hook(lr_getter)
        self.final_lut.set_external_learning_rate_hook(lr_getter)

In [29]:
#lut_net = MultiLutNet(device)
lut_net = LutNet(device)
lut_net

filter finished
conv_lut 0 finished
conv_lut 1 finished
conv_lut 2 finished
conv_lut 3 finished


LutNet(
  (filter_lut): LUTLayer(784 inputs, 2048 detectors, 128 outputs, 10 anchors per detector)
  (conv_luts): ModuleList(
    (0-3): 4 x LUTLayer(128 inputs, 128 detectors, 128 outputs, 15 anchors per detector)
  )
  (final_lut): LUTLayer(128 inputs, 128 detectors, 10 outputs, 15 anchors per detector)
)

In [30]:
from torch.optim.lr_scheduler import LambdaLR

def lr_func(t):
    return min(
        1.0 / (1 + t)**0.5,
        (t / 4000.0) / 4000.0**0.5
    )

# def lr_func(t):
#     return 0.0001

optimizer = optim.SGD(lut_net.parameters(), lr=1.0)
sched = LambdaLR(optimizer, lr_lambda=lr_func)
lut_net.setup_external_learning_rate_hook(optimizer)
# optimizer = optim.Adam(lut_net.parameters(), lr=0.001)

In [34]:
n_epochs = 16
batch_size = 128
with torch.cuda.device(device): 
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(n_epochs):
        lut_net.train()
        train_loader = torch.utils.data.DataLoader(mnist_train_dataset, batch_size=batch_size, shuffle=True)
        correct = 0
        pbar = tqdm(total=len(train_loader))
        for batch_idx, (data, target) in enumerate(train_loader):
            target_one_hot = torch.nn.functional.one_hot(target, num_classes=10)
            optimizer.zero_grad()
            output = lut_net(data)
            loss = loss_func(output, target)
            loss.backward()
            optimizer.step()
            sched.step()
            pbar.update(1)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            if (batch_idx % 100) == 0:
                pbar.set_description(
                    f"Epoch {epoch + 1}/{n_epochs}"
                    f", lr {sched.get_last_lr()[0]:.4f}"
                )
            
        train_acc = 100. * correct / len(train_loader.dataset)

        lut_net.eval()
        correct = 0
        with torch.no_grad():
            test_loader = torch.utils.data.DataLoader(mnist_test_dataset, batch_size=128, shuffle=True)
            for data, target in test_loader:
                output = lut_net(data)
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()
        test_acc = 100. * correct / len(test_loader.dataset)
        print(f'Epoch {epoch+1} Train/test accuracy: {train_acc:.2f}/{test_acc:.2f}, lr {sched.get_last_lr()[0]:.4f}')

  0%|          | 0/469 [00:00<?, ?it/s]

Epoch 1 Train/test accuracy: 99.99/97.63, lr 0.0112


  0%|          | 0/469 [00:00<?, ?it/s]

Epoch 2 Train/test accuracy: 99.99/97.69, lr 0.0109


  0%|          | 0/469 [00:00<?, ?it/s]

...

Epoch 14 Train/test accuracy: 100.00/97.70, lr 0.0084


  0%|          | 0/469 [00:00<?, ?it/s]

Epoch 15 Train/test accuracy: 100.00/97.68, lr 0.0083


  0%|          | 0/469 [00:00<?, ?it/s]

Epoch 16 Train/test accuracy: 100.00/97.70, lr 0.0082


In [32]:
print(lut_net.filter_lut.get_profiling_stats())

profiler is disabled


In [33]:
print(lut_net.conv_luts[0].get_profiling_stats())

profiler is disabled


In [ ]:
print(lut_net.conv_luts[-1].get_profiling_stats())

# JUNK

## Explicit connections LUT (for performance test)

In [ ]:
class LutNet(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.filter_lut = Conv2DLUTLayer(
            input_shape=(28, 28),
            n_anchors_per_detector=10,
            detectors_shape=(32, 32),
            output_kernel_shape=(12, 12),
            receptive_field_shape=(28, 28),
            receptive_field_stride_shape=(28, 28),
            lut_receptive_field_shape=(32, 32),
            lut_receptive_field_stride_shape=(32, 32),
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            synapse_meta=synapse_meta,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            _forward_group_size=32,
            _backward_group_size=32,
            _max_groups_in_growth_buffer=10**7,
            device=device
        )
        print('filter finished')
        l = []
        for i in range(10):
            l.append(
                Conv2DLUTLayer(
                    input_shape=(12, 12),
                    n_anchors_per_detector=14,
                    detectors_shape=(12, 12),
                    output_kernel_shape=(12, 12),
                    sequence_length=1,
                    receptive_field_shape=(12, 12),
                    receptive_field_stride_shape=(12, 12),
                    lut_receptive_field_shape=(12, 12),
                    lut_receptive_field_stride_shape=(12, 12),
                    weights_gradient_policy=g_policy,
                    shared_context=shared_lut_ctx,
                    synapse_meta=synapse_meta,
                    summation_dtype=summation_dtype,
                    random_seed=random_seed,
                    _forward_group_size=32,
                    _backward_group_size=32,
                    device=device
                )
            )
            print(f'conv_lut {i} finished')
        self.conv_luts = nn.ModuleList(l)
        self.final_lut = Conv2DLUTLayer(
            input_shape=(12, 12),
            n_anchors_per_detector=14,
            detectors_shape=(12, 12),
            output_kernel_shape=(1, 10),
            sequence_length=1,
            receptive_field_shape=(12, 12),
            receptive_field_stride_shape=(12, 12),
            lut_receptive_field_shape=(12, 12),
            lut_receptive_field_stride_shape=(12, 12),
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            synapse_meta=synapse_meta,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            _forward_group_size=32,
            _backward_group_size=32,
            device=device
        ) 

    def forward(self, x):
        B = x.shape[0]
        x = self.filter_lut(x.reshape(B, 1, 28, 28))
        for c_lut in self.conv_luts:
            x = c_lut(x) + x
        return self.final_lut(x).reshape(B, 10)
    
    def setup_external_learning_rate_hook(self, optimizer):
        lr_getter = make_lr_getter(optimizer)
        self.filter_lut.set_external_learning_rate_hook(lr_getter)
        for c_lut in self.conv_luts:
            c_lut.set_external_learning_rate_hook(lr_getter)
        self.final_lut.set_external_learning_rate_hook(lr_getter)

# Profiling

In [ ]:
with torch.cuda.device(device):
    with torch.autograd.profiler.profile(use_cuda=True) as prof:
        with torch.profiler.record_function("Dense to sparse conversion"):
            torch.cuda.synchronize(device)
            t0 = time.time()
            indices, values = ds_conv_new.dense_to_sparse_32(
                t, True, 
                shared_context.get_densify_buffers(5000000, torch.device(device))
            )
            torch.cuda.synchronize(device)
            print((time.time() - t0) * 1000, "ms")
            t0 = time.time()
            st = torch.sparse_coo_tensor(
                indices=indices.unsqueeze(0),
                values=values,
                size=indices.shape,
                is_coalesced=True
            )
            print((time.time() - t0) * 1000, "ms")
            torch.cuda.synchronize(device)
            print(ds_conv_new.get_profiling_stats())
print(prof.key_averages().table(sort_by="cuda_time_total"))

In [ ]:
from spiky.util.torch_utils import DenseToSparseConverter
from spiky.lut.LUTLayer import LUTSharedContext


with torch.cuda.device(device):
    torch.cuda.synchronize(device)
    t0 = time.time()
    indices, values = ds_conv_new.dense_to_sparse_32(
        t, True, 
        shared_context.get_densify_buffers(5000000, torch.device(device))
    )
    torch.cuda.synchronize(device)
    print((time.time() - t0) * 1000, "ms")
    t0 = time.time()
    st = torch.sparse_coo_tensor(
        indices=indices.unsqueeze(0),
        values=values,
        size=indices.shape,
        is_coalesced=True
    )
    print((time.time() - t0) * 1000, "ms")
    torch.cuda.synchronize(device)
    print(ds_conv_new.get_profiling_stats())

In [ ]:
fwd_total = 0.0
bwd_total = 0.0
opt_total = 0.0
count = 0

with torch.cuda.device(device): 
    start_fwd = torch.cuda.Event(enable_timing=True)
    end_fwd   = torch.cuda.Event(enable_timing=True)
    start_bwd = torch.cuda.Event(enable_timing=True)
    end_bwd   = torch.cuda.Event(enable_timing=True)
    start_opt = torch.cuda.Event(enable_timing=True)
    end_opt   = torch.cuda.Event(enable_timing=True)

In [ ]:
# with torch.autograd.profiler.profile(use_cuda=True) as prof:
with torch.cuda.device(device): 
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(1):
        lut_net.train()
        train_loader = torch.utils.data.DataLoader(mnist_train_dataset, batch_size=64, shuffle=True)
        correct = 0
        for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
            target_one_hot = torch.nn.functional.one_hot(target, num_classes=10)
            count += 1

            optimizer.zero_grad()

            # forward
            start_fwd.record()
            output = lut_net(data)
            end_fwd.record()
            torch.cuda.synchronize()
            fwd_ms = start_fwd.elapsed_time(end_fwd)
            fwd_total += fwd_ms

            # backward
            loss = loss_func(output, target)
            start_bwd.record()
            loss.backward()
            end_bwd.record()
            torch.cuda.synchronize()
            bwd_ms = start_bwd.elapsed_time(end_bwd)
            bwd_total += bwd_ms

            # optimizer
            start_opt.record()
            optimizer.step()
            end_opt.record()
            torch.cuda.synchronize()
            opt_ms = start_opt.elapsed_time(end_opt)
            opt_total += opt_ms

            sched.step()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            if batch_idx == 128:
                break
        train_acc = 100. * correct / len(train_loader.dataset)
        break
        lut_net.eval()
        correct = 0
        with torch.no_grad():
            test_loader = torch.utils.data.DataLoader(mnist_test_dataset, batch_size=256, shuffle=True)
            for data, target in test_loader:
                output = lut_net(data)
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()
        test_acc = 100. * correct / len(test_loader.dataset)
        print(f'Epoch {epoch+1} Train/test accuracy: {train_acc:.2f}/{test_acc:.2f}, lr {sched.get_last_lr()[0]:.4f}')

In [ ]:
avg_fwd = fwd_total / count
avg_bwd = bwd_total / count
avg_opt = opt_total / count
print(f"Avg fwd/bwd/opt: {avg_fwd:.3f} / {avg_bwd:.3f} / {avg_opt:.3f} ms")

In [ ]:
print(prof.key_averages().table(sort_by="cuda_time_total"))

In [ ]:
print(lut_net.filter_lut.get_profiling_stats())

In [ ]:
print(lut_net.conv_luts[6].get_profiling_stats())

In [ ]:
print(lut_net.final_lut.get_profiling_stats())

In [ ]:
from spiky.lut.LUTLayer import MultiLUT
class MultiLutNet(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.filter_lut = LUTLayer(
            n_inputs=28 * 28,
            n_anchors_per_detector=10,
            n_detectors=1024,
            n_outputs=128,
            synapse_meta=synapse_meta,
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            summation_dtype=summation_dtype,
            # _explicit_anchors=torch.arange(0, 20480, dtype=torch.int32), # for hyperplanes
            random_seed=random_seed,
            device=device
        )
        print('filter finished')
        l = []
        for i in range(10):
            heads = []
            for j in range(4):
                heads.append(
                    LUTLayer(
                        n_inputs=128,
                        n_anchors_per_detector=10,
                        n_detectors=128,
                        n_outputs=128,
                        synapse_meta=synapse_meta,
                        weights_gradient_policy=g_policy,
                        shared_context=shared_lut_ctx,
                        summation_dtype=summation_dtype,
                        random_seed=random_seed,
                        device=device
                    )
                )
                
            l.append(MultiLUT(heads))
            
            print(f'multi_lut {i} finished')
        self.conv_luts = nn.ModuleList(l)
        self.final_lut = LUTLayer(
            n_inputs=128,
            n_anchors_per_detector=16,
            n_detectors=128,
            n_outputs=10,
            synapse_meta=synapse_meta,
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            device=device
        ) 

    def forward(self, x):
        B = x.shape[0]
        x = self.filter_lut(x.reshape(B, 1, 28 * 28))
        for m_lut in self.conv_luts:
            x = m_lut(x) + x
        return self.final_lut(x).squeeze(1)
    
    def setup_external_learning_rate_hook(self, optimizer):
        lr_getter = make_lr_getter(optimizer)
        self.filter_lut.set_external_learning_rate_hook(lr_getter)
        for m_lut in self.conv_luts:
            m_lut.set_external_learning_rate_hook(lr_getter)
        self.final_lut.set_external_learning_rate_hook(lr_getter)

In [ ]:
65536 * 2 * 2 * 6 * 6 * 9

In [ ]:
65536 * 4 * 4 * 18 * 18

In [ ]:
65536 * 4 * 4 * 32